# Aplicación para hacer Web Scrapping de un sitio web y utilización de LLM para resumen y puntos clave

The Scenario
Our company's leadership team, working in the agri-business sector, wants to understand the key dynamics of the global produce trade to identify potential market opportunities. They want a data-driven snapshot of the topics and trends being discussed across several key business areas.
Your mission is to analyze articles from the fresh produce industry's main resources page, extract key information, use modern AI tools to enrich the data, and present your findings in a clear and concise manner.

## Web Scrapping

### Extracción de subdominios.

Primero se va a extraer el contenido de una página web para después generar el algoritmo para extraer los subdominios para en un posterior paso extraer su contenido

URLS a extraer:
- https://www.freshproduce.com/resources/global-trade/
- https://www.freshproduce.com/resources/technology/
- https://www.freshproduce.com/resources/food-safety/

In [1]:
primera_url_a_extraer_subdominio_ejemplo = "https://www.freshproduce.com/resources/global-trade/"

In [2]:
import requests
from bs4 import BeautifulSoup

def scrape_website(url):
    # Realiza web scraping en la URL proporcionada. Extrae todo el HTML.

    try:
        print(f"Extrayendo HTML de: {url}")
        response = requests.get(url) # Realizar una solicitud HTTP GET a la URL
        response.raise_for_status()  # Lanza un error para códigos de estado HTTP incorrectos (4xx o 5xx)

        # 2. Convierte el contenido obtenido a una estructura HTML con BeautifulSoup
        soup_html = BeautifulSoup(response.text, 'html.parser')
        print("Contenido HTML parseado exitosamente.")
        return soup_html

    except requests.exceptions.RequestException as e:
        print(f"Error al conectar con la URL: {e}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")



In [3]:
Extraccion_HTML = scrape_website(primera_url_a_extraer_subdominio_ejemplo)

Extrayendo HTML de: https://www.freshproduce.com/resources/global-trade/
Contenido HTML parseado exitosamente.


Haciendo inspección del sitio web, hay dos clases de etiqueta tipo div necesarias para una correcta inspección:
- search-stats: permite saber el número total de elementos que contiene la página y el número de elementos recorridos
- result-panel: contiene todos los resultados de los artículos, con su respectivo subdominio.

In [4]:
etiquetas_a_extraer = ['search-stats', 'result-panel']
for etiqueta in etiquetas_a_extraer:
    elementos = Extraccion_HTML.find_all(class_=etiqueta)
    print(elementos)

[]
[]


En el paso anterior se puede observar que no encontró ninguna de las etiquetas a buscar. Muy seguramente porque sea contenido dinámico.

Se utilizará Selenium apoyado con Beautiful Soup para casos de contenido dinámico.

In [5]:
from selenium import webdriver #Controlar google chrome para obtener HTML con contenido dinámico
from selenium.webdriver.chrome.options import Options #Controlar google chrome sin interfaz gráfica
from selenium.webdriver.support.ui import WebDriverWait  # Para esperar elementos dinámicos
from selenium.webdriver.support import expected_conditions as EC  # Condiciones para la espera
from selenium.webdriver.common.by import By  # Para localizar elementos (por clase, ID, etc.)
from bs4 import BeautifulSoup # #Para parsear el HTML obtenido con selenium
import chromedriver_autoinstaller #Instalación automática del driver de Chrome

def scrape_website_with_dynamic_content(page, classes_to_wait = None):
    # Instala automáticamente el chromedriver compatible con la versión de Chrome instalada
    chromedriver_autoinstaller.install() 

    options = Options() # Configuraciones de opciones para el navegador
    options.add_argument("--headless") # Ejecuta chrome sin interfaz gráfica

    #Necesario por falta de permisos en el entorno aislado del contenedor
    options.add_argument("--no-sandbox") #Desactiva seguridad de Chrome por problemas de permisos en algunos sistemas
    
    options.add_argument("--disable-dev-shm-usage") #Prevenir fallos por falta de memoria (RAM) compartida en algunos sistemas. Usa memoria de disco duro.

    driver = webdriver.Chrome(options=options) # Inicia navegador chrome con las opciones configuradas

    driver.get(page) # Carga la página web en el navegador

    
    if classes_to_wait:
        tiempo_espera = 30 # Tiempo máximo de espera en segundos para que se cargue el contenido dinámico
        wait = WebDriverWait(driver, tiempo_espera) 
        for class_to_wait in classes_to_wait: #Recorre cada clase a esperar
            wait.until(
                EC.presence_of_element_located((By.CLASS_NAME, class_to_wait))
            )# Espera cada una de las clases


    html_completo = driver.page_source  # Obtención del HTML renderizado completo

    # Usar BeautifulSoup para parsear
    soup = BeautifulSoup(html_completo, "html.parser")
    # Por ejemplo, buscar la tabla con clase 'search-results'
    driver.quit() #Cerrar navegador de Chrome
    return soup

In [6]:
Extraccion_HTML_dinamico = scrape_website_with_dynamic_content(primera_url_a_extraer_subdominio_ejemplo, classes_to_wait=etiquetas_a_extraer)

Verificamos que las etiquetas ahora se encuentren en el contenido.

In [7]:
for etiqueta in etiquetas_a_extraer:
    elementos = Extraccion_HTML_dinamico.find_all(class_=etiqueta)
    print(elementos)
    print(f"Cantidad de elementos encontrados con la clase '{etiqueta}': {len(elementos)}")

[<div class="search-stats"><p>Showing 1 - 9 of 23 results </p></div>]
Cantidad de elementos encontrados con la clase 'search-stats': 1
[<div class="result-panel"><div class="tile eventlandingpage theme-agave tile-grid"><div class="image-wrapper bg-agave"><img alt="Nighttime skyline in Sao Paulo Brazil " loading="eager" src="https://www.freshproduce.com/siteassets/images/events/brazil-conference/sao-paulo-skyline-at-night.jpg/_croppings/event-landing-page-grid.jpg"/></div><div class="caption"><p class="eyebrow">August 6-7, 2025 | São Paulo, Brazil</p><p class="title">The Brazil Conference &amp; Expo</p><p class="description">The Brazil Conference &amp; Expo the meeting point for retail buyers, executives &amp; industry leaders dedicated to the fresh produce supply chain in Brazil.</p><div class="cta-area"><a class="score-button seed-link" href="/events/the-brazil-conference/" target="_self">learn more</a></div></div></div><div class="tile resourcedetailpage theme-mint tile-grid"><div cl

#### Se encontraron los datos a buscar. Ahora se manipularán los datos de las etiquetas a necesitar para sus debidos datos: 'search-stats' y 'result-panel'

##### **Search-stats:** Sirve para ver en qué artículo está actualmente y el número de artículos

In [8]:
search_stats = Extraccion_HTML_dinamico.find(class_=etiquetas_a_extraer[0])  # Buscar el primer elemento con la clase 'search-stats'
if search_stats:
    print("Contenido de 'search-stats':")
    print(search_stats.text)
    print(search_stats.text.split())

Contenido de 'search-stats':
Showing 1 - 9 of 23 results 
['Showing', '1', '-', '9', 'of', '23', 'results']


Al separar todas las palabras hay 3 palabras que nos interesan:
- El último elemento de la página web ('9')
- El número de elementos de la página web ('23')
- Un punto de referencia entre ambos ('of')

Extraeremos cada uno de ellos

In [9]:
search_stats_arreglo = search_stats.text.split() 
print(search_stats_arreglo)
indice_palabra_of = search_stats_arreglo.index("of") #Desde qué elemento está el punto de referencia
print(f"Índice de la palabra 'of': {indice_palabra_of}")
ultimo_elemento_de_pagina = int(search_stats_arreglo[indice_palabra_of -1]) # La palabra anterior a "of" es el último elemento de la página
print(f"Último elemento de la página: {ultimo_elemento_de_pagina}")
total_elementos_de_pagina = int(search_stats_arreglo[indice_palabra_of + 1]) # La palabra siguiente a "of" es el total de elementos de la página
print(f"Total de elementos de la página: {total_elementos_de_pagina}")

['Showing', '1', '-', '9', 'of', '23', 'results']
Índice de la palabra 'of': 4
Último elemento de la página: 9
Total de elementos de la página: 23


Con esto se puede hacer una función que verifique si se ha recorrido todo el contenido.

In [10]:
# Verifica si la página HTML indica que es la última página de resultados.
def bool_is_final_page(HTML, etiqueta='search-stats'):
    search_stats = HTML.find(class_=etiqueta)
    if search_stats:
        search_stats_arreglo = search_stats.text.split()
        indice_palabra_of = search_stats_arreglo.index("of")
        ultimo_elemento_de_pagina = int(search_stats_arreglo[indice_palabra_of - 1])
        total_elementos_de_pagina = int(search_stats_arreglo[indice_palabra_of + 1])
        return (ultimo_elemento_de_pagina >= total_elementos_de_pagina) #No debería haber más elementos en la página web, pero el método == puede fallar si hay un error en la web.
    return True # Si no se encuentra la clase, asumimos que es la última página para evitar errores en el bucle de paginación.

In [11]:
#Uso de la función para verificar
ultima_pagina = bool_is_final_page(Extraccion_HTML_dinamico, etiquetas_a_extraer[0])
print(f"¿Es la última página? {ultima_pagina}")

¿Es la última página? False


##### **Result panel:** Contiene todos los subdominios de los articulos

In [12]:
result_panel = Extraccion_HTML_dinamico.find(class_=etiquetas_a_extraer[1])  # Buscar el primer elemento con la clase 'search-stats' (solo existe uno)
if result_panel:
    print(f"Cantidad de elementos encontrados con la clase '{etiquetas_a_extraer[1]}': {len(result_panel)}")
    print("Contenido de 'search-stats':")
    print(result_panel)
    print("-----")
    for elemento in result_panel:
        print(elemento)
        print("-----")

Cantidad de elementos encontrados con la clase 'result-panel': 9
Contenido de 'search-stats':
<div class="result-panel"><div class="tile eventlandingpage theme-agave tile-grid"><div class="image-wrapper bg-agave"><img alt="Nighttime skyline in Sao Paulo Brazil " loading="eager" src="https://www.freshproduce.com/siteassets/images/events/brazil-conference/sao-paulo-skyline-at-night.jpg/_croppings/event-landing-page-grid.jpg"/></div><div class="caption"><p class="eyebrow">August 6-7, 2025 | São Paulo, Brazil</p><p class="title">The Brazil Conference &amp; Expo</p><p class="description">The Brazil Conference &amp; Expo the meeting point for retail buyers, executives &amp; industry leaders dedicated to the fresh produce supply chain in Brazil.</p><div class="cta-area"><a class="score-button seed-link" href="/events/the-brazil-conference/" target="_self">learn more</a></div></div></div><div class="tile resourcedetailpage theme-mint tile-grid"><div class="image-wrapper bg-mint"><img alt='Bask

Al inspeccionar cada elemento del articulo, cada uno contiene un botón con el hipervinculo en la clase "cta-area", por lo que ahora extraeremos el contenido solamente en "cta-area". Se buscará la etiqueta `<a>` y dentro de ella el hipervinculo.

In [13]:
encontrar_todos_los_hipervinculos = result_panel.find_all('div', class_='cta-area')  # Buscar todos los hipervínculos en el HTML
print(encontrar_todos_los_hipervinculos)
todos_los_subdominios = []  # Lista para almacenar los subdominios encontrados
for enlace in encontrar_todos_los_hipervinculos:
    a_tag = enlace.find('a')  # Buscar la etiqueta <a> dentro de cada div
    #print(a_tag)
    if a_tag and 'href' in a_tag.attrs:  # Verificar si existe el atributo href
        print(f"Enlace encontrado: {a_tag['href']}")  # Imprimir el enlace
        todos_los_subdominios.append(a_tag['href'])
    else:
        print("No se encontró un enlace válido en este elemento.")

[<div class="cta-area"><a class="score-button seed-link" href="/events/the-brazil-conference/" target="_self">learn more</a></div>, <div class="cta-area"><a class="score-button seed-link" href="/resources/consumer-trends/americans-and-sustainability/" target="_self">Learn More</a></div>, <div class="cta-area"><a class="score-button seed-link" href="/events/the-mexico-conference/" target="_self">Learn more</a></div>, <div class="cta-area"><a class="score-button seed-link" href="/resources/advocacy/trumps-first-100-days/" target="_self">Learn More</a></div>, <div class="cta-area"><a class="score-button seed-link" href="/resources/global-trade/tariff-resources/" target="_self">Learn More</a></div>, <div class="cta-area"><a class="score-button seed-link" href="/resources/Commodities/international-retail-point-of-sale-data/" target="_self">Learn More</a></div>, <div class="cta-area"><a class="score-button seed-link" href="/resources/supply-chain-management/global-market-access/" target="_se

Al utilizar el dominio principal (https://www.freshproduce.com) junto al subdominio podemos acceder a las páginas web con su respectivo contenido.

In [14]:
dominio_principal = "https://www.freshproduce.com"
subdominio_ejemplo = todos_los_subdominios[0]

print(f"Link hacia el articulo: {dominio_principal}{subdominio_ejemplo}")

Link hacia el articulo: https://www.freshproduce.com/events/the-brazil-conference/


Con ello podemos hacer un algoritmo que permita tomar desde el HTML todos los subdominios de la página web

In [15]:
def obtener_subdominios(HTML, etiqueta_completa_todos_elementos = 'result-panel' ,etiqueta_elemento_individual='cta-area', dominio_principal="https://www.freshproduce.com"):
    result_panel_subdominio = HTML.find(class_=etiqueta_completa_todos_elementos) 
    
    if not result_panel_subdominio:
        print(f"No se encontró la clase '{etiqueta}' en el HTML.")
        return []
    subdominios = []
    encontrar_todos_los_hipervinculos = result_panel_subdominio.find_all('div', class_=etiqueta_elemento_individual)
    for enlace in encontrar_todos_los_hipervinculos:
        a_tag = enlace.find('a')
        if a_tag and 'href' in a_tag.attrs:
            subdominio_completo = f"{dominio_principal}{a_tag['href']}"
            subdominios.append(subdominio_completo)
    
    return subdominios

In [16]:
ejemplo_subdominios = obtener_subdominios(Extraccion_HTML_dinamico)
print("Subdominios encontrados:")
for subdominio in ejemplo_subdominios:
    print(subdominio)

Subdominios encontrados:
https://www.freshproduce.com/events/the-brazil-conference/
https://www.freshproduce.com/resources/consumer-trends/americans-and-sustainability/
https://www.freshproduce.com/events/the-mexico-conference/
https://www.freshproduce.com/resources/advocacy/trumps-first-100-days/
https://www.freshproduce.com/resources/global-trade/tariff-resources/
https://www.freshproduce.com/resources/Commodities/international-retail-point-of-sale-data/
https://www.freshproduce.com/resources/supply-chain-management/global-market-access/
https://www.freshproduce.com/resources/global-trade/mexico-exports-to-us/
https://www.freshproduce.com/resources/Commodities/global-table-grape-shipment-data/


#### Navegar por el numero de paginas del sitio web.

Al usar el dominio https://www.freshproduce.com/resources/global-trade/ y navegar por la página, este cambia a https://www.freshproduce.com/resources/global-trade/?pageNumber=0 y https://www.freshproduce.com/resources/global-trade/?pageNumber=1 en su segunda página, por lo que asumimos que para cambiar de página utilizaremos esto:

`https://www.freshproduce.com/resources/global-trade/?pageNumber=NUMPAGINA`, donde NUMPAGINA es el número de página (iniciando desde 0).

Esto aplica para las demás categorías.

URLS con su respectiva navegación:
- https://www.freshproduce.com/resources/global-trade/?pageNumber=0
- https://www.freshproduce.com/resources/technology/?pageNumber=0
- https://www.freshproduce.com/resources/food-safety/?pageNumber=0

Por lo que el algoritmo para usarlo es:
`URL_BASE/?pageNumber=NUMPAGINA` donde por ejemplo la URL b ase de global-trade es `https://www.freshproduce.com/resources/global-trade` y NUMPAGINA es el número de página

In [17]:
numero_de_paginas_maximo_a_extraer = 2  # Número máximo de páginas a extraer
paginas_a_extraer = []  # Lista para almacenar las URLs de las páginas a extraer
for i in range (0,numero_de_paginas_maximo_a_extraer):
    print(f"Extrayendo página {i + 1} de {numero_de_paginas_maximo_a_extraer}")
    url_pagina = f"{primera_url_a_extraer_subdominio_ejemplo}?pageNumber={i}"
    print(f"URL de la página a extraer: {url_pagina}")
    Extraccion_HTML_dinamico = scrape_website_with_dynamic_content(url_pagina, classes_to_wait=etiquetas_a_extraer)
    
    subdominios_en_pagina = obtener_subdominios(Extraccion_HTML_dinamico)
    for subdominio in subdominios_en_pagina:
        print(subdominio)
        paginas_a_extraer.append(subdominio)  # Agregar la URL de la página a la lista

    if bool_is_final_page(Extraccion_HTML_dinamico, etiquetas_a_extraer[0]):
        print("Se ha alcanzado la última página. Terminando la extracción.")
        break

Extrayendo página 1 de 2
URL de la página a extraer: https://www.freshproduce.com/resources/global-trade/?pageNumber=0
https://www.freshproduce.com/events/the-brazil-conference/
https://www.freshproduce.com/resources/consumer-trends/americans-and-sustainability/
https://www.freshproduce.com/events/the-mexico-conference/
https://www.freshproduce.com/resources/advocacy/trumps-first-100-days/
https://www.freshproduce.com/resources/global-trade/tariff-resources/
https://www.freshproduce.com/resources/Commodities/international-retail-point-of-sale-data/
https://www.freshproduce.com/resources/supply-chain-management/global-market-access/
https://www.freshproduce.com/resources/global-trade/mexico-exports-to-us/
https://www.freshproduce.com/resources/Commodities/global-table-grape-shipment-data/
Extrayendo página 2 de 2
URL de la página a extraer: https://www.freshproduce.com/resources/global-trade/?pageNumber=1
https://www.freshproduce.com/resources/Commodities/top-fresh-produce-commodity-pro

In [18]:
print(paginas_a_extraer)

['https://www.freshproduce.com/events/the-brazil-conference/', 'https://www.freshproduce.com/resources/consumer-trends/americans-and-sustainability/', 'https://www.freshproduce.com/events/the-mexico-conference/', 'https://www.freshproduce.com/resources/advocacy/trumps-first-100-days/', 'https://www.freshproduce.com/resources/global-trade/tariff-resources/', 'https://www.freshproduce.com/resources/Commodities/international-retail-point-of-sale-data/', 'https://www.freshproduce.com/resources/supply-chain-management/global-market-access/', 'https://www.freshproduce.com/resources/global-trade/mexico-exports-to-us/', 'https://www.freshproduce.com/resources/Commodities/global-table-grape-shipment-data/', 'https://www.freshproduce.com/resources/Commodities/top-fresh-produce-commodity-profiles/', 'https://www.freshproduce.com/resources/global-trade/how-to-export-cut-flowers-from-mexico-to-the-us/', 'https://www.freshproduce.com/resources/supply-chain-management/fsma-204-best-practices-webinar/

Con todo lo anterior podemos realizar una extracción de todas las páginas web.

In [19]:
def extraccion_de_subdominios(url_a_extraer, numero_de_paginas_maximo_a_extraer=2, etiquetas_a_extraer_funcion=['search-stats', 'result-panel'], etiqueta_elemento_individual='cta-area', dominio_principal="https://www.freshproduce.com"):
    paginas_a_extraer = []  # Lista para almacenar las URLs de las páginas a extraer
    for (url, etiqueta) in url_a_extraer: #Separacion de URL y etiqueta para la extracción
        for i in range(0,numero_de_paginas_maximo_a_extraer):
            url_pagina = f"{url}?pageNumber={i}"
            print(f"URL de la página actual a extraer: {url_pagina}")
            Extraccion_HTML_dinamico = scrape_website_with_dynamic_content(url_pagina, classes_to_wait=etiquetas_a_extraer_funcion)

            etiqueta_completa_todos_elementos = etiquetas_a_extraer_funcion[1]
            subdominios_en_pagina = obtener_subdominios(Extraccion_HTML_dinamico, etiqueta_completa_todos_elementos,etiqueta_elemento_individual, dominio_principal)
            for subdominio in subdominios_en_pagina:
                #print(subdominio)
                paginas_a_extraer.append([subdominio, etiqueta])  # Agregar la URL de la página a la lista y su etiqueta asociada

            # Verificar si se ha alcanzado la última página
            etiqueta_busqueda_numeroElementos = etiquetas_a_extraer_funcion[0]  
            if bool_is_final_page(Extraccion_HTML_dinamico, etiqueta_busqueda_numeroElementos):
                print("Se ha alcanzado la última página. Terminando la extracción.")
                break
    print("Páginas a extraer final:", len(paginas_a_extraer))
    return paginas_a_extraer

In [20]:
urls_extraer_información = [
                            ["https://www.freshproduce.com/resources/global-trade/","Global Trade"],
                            ["https://www.freshproduce.com/resources/technology/","Technology"],
                            ["https://www.freshproduce.com/resources/food-safety/","Food Safety"]
                            ] # Ya viene con su respectivo etiquetado para identificar el subdominio

In [21]:
paginas_extraidas = extraccion_de_subdominios(urls_extraer_información, 20)

URL de la página actual a extraer: https://www.freshproduce.com/resources/global-trade/?pageNumber=0
URL de la página actual a extraer: https://www.freshproduce.com/resources/global-trade/?pageNumber=1
URL de la página actual a extraer: https://www.freshproduce.com/resources/global-trade/?pageNumber=2
Se ha alcanzado la última página. Terminando la extracción.
URL de la página actual a extraer: https://www.freshproduce.com/resources/technology/?pageNumber=0
URL de la página actual a extraer: https://www.freshproduce.com/resources/technology/?pageNumber=1
URL de la página actual a extraer: https://www.freshproduce.com/resources/technology/?pageNumber=2
URL de la página actual a extraer: https://www.freshproduce.com/resources/technology/?pageNumber=3
URL de la página actual a extraer: https://www.freshproduce.com/resources/technology/?pageNumber=4
URL de la página actual a extraer: https://www.freshproduce.com/resources/technology/?pageNumber=5
URL de la página actual a extraer: https://

In [22]:
print(f"Total de subdominios extraídos: {len(paginas_extraidas)}")

Total de subdominios extraídos: 209


Ahora lo pasamos a un formato Pandas para mejor control de los datos

In [23]:
import pandas as pd
columnas = ['URL','Category']
df_dominios = pd.DataFrame(paginas_extraidas, columns=columnas)

In [ ]:
print(df_dominios.head())

In [26]:
print(df_dominios.Category.describe()) #Ver descripción de las categorías

count            209
unique             3
top       Technology
freq              93
Name: Category, dtype: object


In [27]:
df_dominios.to_csv(f"extracted_subdomains.csv", index=False)  # Guardar el DataFrame en un archivo CSV

### Extracción de información de subdominios.

Teniendo la información de los anteriores subdominios en un .csv vamos a extraer toda la información de cada página web.

## AI-Powered Analysis & Enrichment